In [127]:
# Import dependencies
import pandas as pd
import numpy as np
import sklearn
import category_encoders as ce

In [128]:
# Loads the data into a DataFrame
df = pd.read_csv("../Resources/dataset.csv")

# Removes the csv index (duplicate) column
df = df[['artists', 'track_id', 'album_name','track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'track_genre']]
df.head()    

,artists,track_id,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,Gen Hoshino,5SuOikwiRyPMVoIQDJUgSV,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,Ben Woodward,4qPNDBW1i3p13qLCt0Ki3A,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,Ingrid Michaelson;ZAYN,1iJBSr7s7jYXzM8EGcbK5b,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,Kina Grannis,6lfxq3CG4xtTiEg7opyCyx,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,Chord Overstreet,5vjLSffimiIP26QG5WcN2K,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


# Data value ranges:
* popularity - 0-100
* duration - changed to minutes
* explicit - T/F -> 1/0
* danceability - 0-1
* energy - 0-1
* key - 0-11, dummy data this
* loudness - ~-49-4, probably scale or normalize this
* mode - 0 or 1; major or minor key
* speechiness - 0-1
* acousticness - 0-1
* instrumentalness - 0-1, prediction of not having vocals
* liveliness - 0-1, prediction of whether track was recorded live
* valence - 0-1 - how positive does music sound
* tempo - 0-243, tempo in bpm, probably scale or normalize this
* time signature - 3-7, representing 3/4 - 7/4; dummy data
* track_genre - genre - dummy data this

# Manipulations to perform:
* Remove duplicate rows (DONE)
* Separate out multiple artists (DONE)
  * Determine primary artist
  * Determine number of artists
  * Bin number of artists
    * Dummy data the bins
* Duration changes (DONE)
  * Remove duration = 0 
  * Convert duration to minutes
  * Bin duration by length (minutes)
    * Dummy data the bins
* Change explicit into 0/1
  * 1 = explicit (True)
  * 0 = not explicit (False)
* Separate out genre, time_signature, key, and binned values (number of artists and duration) into separate columns

* **NOTE**: I have, for the most part, kept all columns. 
  * This means that for all models, you will have to delete all unwanted columns
  * I may make a final filtered version, but you should check on what I've deleted/ kept and can change that for your models
  * This has also not been normalized yet

In [129]:
df.shape

(114000, 20)

In [130]:
df = df.drop_duplicates(ignore_index=True)
df.shape

(113550, 20)

In [131]:
# Remove duration = 0
df=df[df['duration_ms'] != 0]
# Reset the index
df = df.reset_index(drop=True)

# Convert duration to minutes
df['duration_min'] = df['duration_ms'] / 60000

# Bin duration
# Set up a list of bins
duration_bins = [1, 2, 3, 4, 5, 6]
# Set up list of conditions
duration_conditions = [
    (df["duration_min"] < 2), 
    (df["duration_min"] >= 2) & (df["duration_min"] < 3),
    (df["duration_min"] >= 3) & (df["duration_min"] < 4),
    (df["duration_min"] >= 4) & (df["duration_min"] < 5),
    (df["duration_min"] >= 5) & (df["duration_min"] < 6),
    (df["duration_min"] >= 6)
]
# Set up the column with bins
df["duration_binned"] = np.select(duration_conditions, duration_bins)

# Confirm binning
df['duration_binned'].value_counts()

duration_binned
3    42269
2    25955
4    22836
5     8574
6     7737
1     6178
Name: count, dtype: int64

In [132]:
# Determine the number of artists for each track and the number of times it comes up
num_artists = []
count = []
for index, row in df.iterrows():
    num = len(str(row['artists']).split(";"))
    if num not in num_artists:
        num_artists.append(num)
        count.append(1)
    else:
        count[num_artists.index(num)] += 1
print(list(zip(num_artists, count)))

[(1, 83656), (2, 21017), (3, 6188), (4, 1681), (5, 512), (10, 24), (7, 117), (6, 219), (18, 6), (11, 11), (8, 51), (17, 9), (9, 24), (15, 4), (14, 4), (19, 2), (12, 8), (31, 1), (13, 5), (26, 1), (23, 3), (30, 2), (20, 1), (22, 1), (32, 1), (38, 1)]


In [133]:
# Select and keep only primary artist
df["primary_artist"] = df['artists'].str.split(";").str[0]
# Determine the number of artists for each track and make a column for this
df["number_artists"] = df['artists'].apply(lambda x: len(str(x).split(";")))
df.head()

,artists,track_id,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,duration_min,duration_binned,primary_artist,number_artists
0,Gen Hoshino,5SuOikwiRyPMVoIQDJUgSV,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic,3.844433,3,Gen Hoshino,1
1,Ben Woodward,4qPNDBW1i3p13qLCt0Ki3A,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic,2.493500,2,Ben Woodward,1
2,Ingrid Michaelson;ZAYN,1iJBSr7s7jYXzM8EGcbK5b,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic,3.513767,3,Ingrid Michaelson,2
3,Kina Grannis,6lfxq3CG4xtTiEg7opyCyx,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic,3.365550,3,Kina Grannis,1
4,Chord Overstreet,5vjLSffimiIP26QG5WcN2K,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic,3.314217,3,Chord Overstreet,1


In [134]:
artist_counts = df["number_artists"].value_counts()
artist_counts

number_artists
1     83656
2     21017
3      6188
4      1681
5       512
6       219
7       117
8        51
10       24
9        24
11       11
17        9
12        8
18        6
13        5
15        4
14        4
23        3
19        2
30        2
31        1
26        1
20        1
22        1
32        1
38        1
Name: count, dtype: int64

In [135]:
# Bin the number of artists
## Set up a list of numbers of artists to replace - anything with less than 
artist_nums_to_replace = [num for num in artist_counts.index if artist_counts[num] < 500]
## Set up a new column to hold binned data
df["num_artists_binned"] = df['number_artists']
## Replace the numbers in the list with 6, which corresponds to >5 artists
for num in artist_nums_to_replace:
    df['num_artists_binned'] = df['num_artists_binned'].replace(num, 6)
## Check binning
df['num_artists_binned'].value_counts()

num_artists_binned
1    83656
2    21017
3     6188
4     1681
5      512
6      495
Name: count, dtype: int64

In [136]:
# Performs one-hot-encoding on: time_signature, key, binned # of artists, and binned duration
pd.set_option('display.max_columns', None)
encoded_df = pd.get_dummies(df, columns=['time_signature', 'key', 'num_artists_binned', 'duration_binned'])
encoded_df

,artists,track_id,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_genre,duration_min,primary_artist,number_artists,time_signature_0,time_signature_1,time_signature_3,time_signature_4,time_signature_5,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,num_artists_binned_1,num_artists_binned_2,num_artists_binned_3,num_artists_binned_4,num_artists_binned_5,num_artists_binned_6,duration_binned_1,duration_binned_2,duration_binned_3,duration_binned_4,duration_binned_5,duration_binned_6
0,Gen Hoshino,5SuOikwiRyPMVoIQDJUgSV,Comedy,Comedy,73,230666,False,0.676,0.4610,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,acoustic,3.844433,Gen Hoshino,1,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False
1,Ben Woodward,4qPNDBW1i3p13qLCt0Ki3A,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,acoustic,2.493500,Ben Woodward,1,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False
2,Ingrid Michaelson;ZAYN,1iJBSr7s7jYXzM8EGcbK5b,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,acoustic,3.513767,Ingrid Michaelson,2,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False
3,Kina Grannis,6lfxq3CG4xtTiEg7opyCyx,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,acoustic,3.365550,Kina Grannis,1,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False
4,Chord Overstreet,5vjLSffimiIP26QG5WcN2K,Hold On,Hold On,82,198853,False,0.618,0.4430,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,acoustic,3.314217,Chord Overstreet,1,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113544,Rainy Lullaby,2C3TZjDRiAzdyViavDJ217,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,world-music,6.416650,Rainy Lullaby,1,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True
113545,Rainy Lullaby,1hIz5L4IB9hN3WRYPOCGPw,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,world-music,6.416667,Rainy Lullaby,1,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True
113546,Cesária Evora,6x8ZfSoqDjuNa5SVP5QjvX,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,world-music,4.524433,Cesária Evora,1,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False
113547,Michael W. Smith,2e6sXL2bYv4bSz6VTdnfLs,Change Your World,Friends,41,283893,False,0.587,0.5060,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,world-music,4.731550,Michael W. Smith,1,False

In [137]:
# Convert T/F columns to 1/0
encoded_df[['explicit','time_signature_0', 'time_signature_1',
       'time_signature_3', 'time_signature_4', 'time_signature_5', 'key_0',
       'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8',
       'key_9', 'key_10', 'key_11', 'num_artists_binned_1',
       'num_artists_binned_2', 'num_artists_binned_3', 'num_artists_binned_4',
       'num_artists_binned_5', 'num_artists_binned_6', 'duration_binned_1',
       'duration_binned_2', 'duration_binned_3', 'duration_binned_4',
       'duration_binned_5', 'duration_binned_6']] = encoded_df[['explicit','time_signature_0', 'time_signature_1',
       'time_signature_3', 'time_signature_4', 'time_signature_5', 'key_0',
       'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8',
       'key_9', 'key_10', 'key_11', 'num_artists_binned_1',
       'num_artists_binned_2', 'num_artists_binned_3', 'num_artists_binned_4',
       'num_artists_binned_5', 'num_artists_binned_6', 'duration_binned_1',
       'duration_binned_2', 'duration_binned_3', 'duration_binned_4',
       'duration_binned_5', 'duration_binned_6']].astype(int).reset_index(drop=True)
encoded_df.shape

(113549, 50)

In [138]:
# Encodes genres as binary strings in a dataframe
genre_encoder = ce.BinaryEncoder().fit_transform(encoded_df['track_genre']).reset_index(drop=True)
genre_encoder.shape

(113549, 7)

In [139]:
full_df = encoded_df.merge(genre_encoder, left_index=True, right_index=True)
full_df.columns

Index(['artists', 'track_id', 'album_name', 'track_name', 'popularity',
       'duration_ms', 'explicit', 'danceability', 'energy', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'track_genre', 'duration_min', 'primary_artist',
       'number_artists', 'time_signature_0', 'time_signature_1',
       'time_signature_3', 'time_signature_4', 'time_signature_5', 'key_0',
       'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8',
       'key_9', 'key_10', 'key_11', 'num_artists_binned_1',
       'num_artists_binned_2', 'num_artists_binned_3', 'num_artists_binned_4',
       'num_artists_binned_5', 'num_artists_binned_6', 'duration_binned_1',
       'duration_binned_2', 'duration_binned_3', 'duration_binned_4',
       'duration_binned_5', 'duration_binned_6', 'track_genre_0',
       'track_genre_1', 'track_genre_2', 'track_genre_3', 'track_genre_4',
       'track_genre_5', 'track_genre_6'],
      dtype='o

In [140]:
# Filter columns; designate what to keep
filtered_df = full_df[['primary_artist', 'track_name', 'popularity',
       'explicit', 'danceability', 'energy', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_min', 'time_signature_0', 'time_signature_1',
       'time_signature_3', 'time_signature_4', 'time_signature_5', 'key_0',
       'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8',
       'key_9', 'key_10', 'key_11', 'num_artists_binned_1', 'num_artists_binned_2', 'num_artists_binned_3', 'num_artists_binned_4',
       'num_artists_binned_5', 'num_artists_binned_6', 'track_genre_0',
       'track_genre_1', 'track_genre_2', 'track_genre_3', 'track_genre_4',
       'track_genre_5', 'track_genre_6']]
filtered_df.head()

,primary_artist,track_name,popularity,explicit,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_min,time_signature_0,time_signature_1,time_signature_3,time_signature_4,time_signature_5,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,num_artists_binned_1,num_artists_binned_2,num_artists_binned_3,num_artists_binned_4,num_artists_binned_5,num_artists_binned_6,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,track_genre_5,track_genre_6
0,Gen Hoshino,Comedy,73,0,0.676,0.4610,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,3.844433,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,Ben Woodward,Ghost - Acoustic,55,0,0.420,0.1660,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,2.493500,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,Ingrid Michaelson,To Begin Again,57,0,0.438,0.3590,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,3.513767,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
3,Kina Grannis,Can't Help Falling In Love,71,0,0.266,0.0596,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3.365550,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,Chord Overstreet,Hold On,82,0,0.618,0.4430,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,3.314217,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [141]:
# Remove commas from 'track_name' column, for use with spark
filtered_df['track_name'] = filtered_df['track_name'].str.replace(",","")

C:\Users\kronh\AppData\Local\Temp\ipykernel_25388\2999912686.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['track_name'] = filtered_df['track_name'].str.replace(",","")


In [142]:
filtered_df.to_csv("../Resources/filtered_encoded_dataset.csv", index=False)